# 4.  Image Classification
### We will build a Convolutional Neural Network (CNN) to classify 
### handwritten digits from 0 to 1 using the MNIST dataset.

In [1]:
# import the necessary libraries for this task
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy

In [2]:
# function to count number of parameters
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

### Load the Dataset (MNIST)

In [3]:
# images are 28x28 pixels
input_size  = 28*28   

# there are 10 classes (digits of 0 to 9)
output_size = 10      

# Load and transform training and test data
train_dataset = datasets.MNIST(root='./data', download=True,
                            train=True, transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))]))

test_dataset = datasets.MNIST(root='./data', 
                           train=False, transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))]))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=64, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=1000, 
                                          shuffle=False)

###  4.1  New Model Class

In [4]:
# We create a model with two convolution layers and one fully connected layer at the end
## Perform activations using  Rectified linear Unit (Relu) and Max-pooling or
## Average pooling after each convolution. Finally, add a soft(arg)max after the fully 
## connected layer.

class CNN(nn.Module):
    def __init__(self, input_size, conv_kernel, pooling_kernel, stride_size, zero_padding, max_pooling):
        super(CNN, self).__init__()
        
        # First Convolution
        if zero_padding is True:
            self.conv_1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=conv_kernel, stride=stride_size, padding=2)
        else:
            self.conv_1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=conv_kernel, stride=stride_size, padding=0)
        self.relu_1 = nn.ReLU()
        
        
        # First Pooling
        if max_pooling is True:
            # Perform Max pooling
            self.pool_1 = nn.MaxPool2d(kernel_size=pooling_kernel, stride=stride_size)
        else:
            # Perform Average pooling
            self.pool_1 = nn.AvgPool2d(kernel_size=pooling_kernel, stride=stride_size)
            
        # Second Convolution
        if zero_padding is True:
            self.conv_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=conv_kernel, stride=stride_size, padding=2)
        else:
            self.conv_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=conv_kernel, stride=stride_size, padding=0)
        self.relu_2 = nn.ReLU()
        
        # Second Pooling
        if max_pooling is True:
            # Perform Max pooling
            self.pool_2 = nn.MaxPool2d(kernel_size=pooling_kernel, stride=stride_size)
        else:
            # Perform Average pooling
            self.pool_2 = nn.AvgPool2d(kernel_size=pooling_kernel, stride=stride_size)
        
        # Fully Connected Layer
        if zero_padding is True:
            self.fc = nn.Linear(32 * 7 * 7, 10)
        else:
            self.fc = nn.Linear(32 * 4 * 4, 10)
        
        self.soft_arg_max = nn.LogSoftmax()
        
    def forward(self, x):
        # First Convolution
        output = self.conv_1(x)
        output = self.relu_1(output)
        
        # Pooling
        output = self.pool_1(output)
        
        # Second Convolution 
        output = self.conv_2(output)
        output = self.relu_2(output)
        
        # Pooling 
        output = self.pool_2(output)
        
        # Resize
        output = output.view(output.size(0), -1)

        # Dense
        output = self.fc(output)   
        output = self.soft_arg_max(output, dim=1)
        
        return output

### 4.2   Zero Padding Model Tensors
Instantiate Model Class

In [5]:
model_1 = CNN(input_size, conv_kernel=5, pooling_kernel=2,  stride_size=1, zero_padding=True, max_pooling=True)
model_1

CNN(
  (conv_1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu_1): ReLU()
  (pool_1): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv_2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu_2): ReLU()
  (pool_2): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=1568, out_features=10, bias=True)
  (soft_arg_max): LogSoftmax()
)

### Parameters

In [6]:
# Input parameter sizes
params = list(model_1.parameters())
print(len(params))

for i in range(len(params)):
    print(params[i].size())

# Total number of parameters
print('Number of parameters: {}'.format(get_n_params(model_1)))

6
torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 1568])
torch.Size([10])
Number of parameters: 28938


### 4.3   Valid Padding Model Tensors
Instantiate Model Class


In [7]:
model_2 = CNN(input_size, conv_kernel=5, pooling_kernel=2,  stride_size=1, zero_padding=False, max_pooling=True)
model_2

CNN(
  (conv_1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu_1): ReLU()
  (pool_1): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv_2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (relu_2): ReLU()
  (pool_2): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=512, out_features=10, bias=True)
  (soft_arg_max): LogSoftmax()
)

### Parameters

In [8]:
# Input parameter sizes
params = list(model_2.parameters())
print(len(params))

for i in range(len(params)):
    print(params[i].size())

# Total number of parameters
print('Number of parameters: {}'.format(get_n_params(model_2)))

6
torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 512])
torch.Size([10])
Number of parameters: 18378
